In [1]:
import pandas as pd
import csv
import os
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
try:
    os.makedirs("assets/annotated-corpus/train")
except:
    pass
try:
    os.makedirs("assets/annotated-corpus/test")
except:
    pass

In [3]:
# Загрузка nltk ресурсов
nltk.download('punkt')
nltk.download('wordnet')

# Инициализация стеммера и лемматизатора
stemmer = SnowballStemmer(language='english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [21]:
def reg(text):
    # Регулярные выражения для различных случаев
  phone_pattern = r'\+?[78]?[-.\s]?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{2}[-.\s]?\d{2}'
  address_pattern = r'\b(?:blv\.|st\.|ave\.|app\.)[\s.,]*\S+'
  email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
  emoji_pattern = r'[\w\s]+(?::|;|X|\^)(?:-)?(?:\)|\(|D|P)'
  math_pattern = r'[a-zA-Z0-9\*\+\-\=\/\(\)\[\]\{\}\^\%\$\@\#\~\,]+'
  abbreviation_pattern = r'\b[A-Z][a-z]*\.'

  # Комбинированное регулярное выражение для токенизации
  combined_pattern = "|".join((phone_pattern, address_pattern, email_pattern, emoji_pattern, math_pattern, abbreviation_pattern, "\w]+"))
  tokens = re.findall(combined_pattern, text)
  return tokens

In [26]:
text = "+79112451944 aaaaa ave. Street 17 my@perfect.email aaaaaaa :D X) (943%) USA GmBH"
reg(text)

['+79112451944',
 'aaaaa',
 'ave. Street',
 '17',
 'my@perfect.email',
 ' aaaaaaa :D',
 ' X)',
 '(943%)',
 'USA',
 'GmBH']

In [23]:
def stem(data, subset):
    all_sentences = data.values.tolist()

    for i in range(len(all_sentences)):
        el = all_sentences[i]
        text = el[1] + " " + el[2]
        label = el[0]

        sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z0-9])', text)
        for sentence in sentences:
            tokens = reg(sentence)
            for token in tokens:
                with open(f'assets/annotated-corpus/{subset}/{label}.tsv', 'a', newline='') as tsvfile:
                    writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
                    writer.writerow([i, token, stemmer.stem(token), lemmatizer.lemmatize(token)])
            with open(f'assets/annotated-corpus/{subset}/{label}.tsv', 'a', newline='') as tsvfile:
                writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
                writer.writerow([i, "<endofsentence>", "<endofsentence>", "<endofsentence>"])


In [24]:
stem(pd.read_csv("https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv", header=None), "train")

In [25]:
stem(pd.read_csv("https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/test.csv", header=None), "test")